In this lesson, we trained a linear neural network to distinguish background green pixels from other foreground pixels. Do you think the network will converge to the same weights every time it is run? Try it and see. Is there any property of the weights that seems consistent from one run to another?

In [7]:
!wget https://github.com/mlittmancs/great_courses_ml/raw/master/imgs/greenML.png

--2023-09-09 22:08:17--  https://github.com/mlittmancs/great_courses_ml/raw/master/imgs/greenML.png
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/mlittmancs/great_courses_ml/master/imgs/greenML.png [following]
--2023-09-09 22:08:17--  https://raw.githubusercontent.com/mlittmancs/great_courses_ml/master/imgs/greenML.png
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1190198 (1.1M) [image/png]
Saving to: ‘greenML.png’

greenML.png         100%[===================>]   1.13M  --.-KB/s    in 0.05s   

2023-09-09 22:08:17 (20.7 MB/s) - ‘greenML.png’ saved [1190198/1190198]



Download the image.

In [8]:
import numpy as np
from tensorflow import keras

img = keras.utils.load_img("greenML.png")

Trim the edges of the image.

In [9]:
arr = image.img_to_array(img)
# Trim off edges
arr = arr[:697,:]

Isolate out the bacground and convert it to a dataset of positive examples, `yesList`.

In [10]:
# background
tmp = arr[:,:360]

yesList = np.reshape(tmp,(-1,3))

Print the number of values in each dimension of `tmp`.

Isolate out the foreground and make a dataset of foreground pixels called `noList`.

In [11]:
# foreground
tmp = arr[30:,547:620]

noList = np.reshape(tmp,(-1,3))

Finalize our dataset, with a variable `alldat`, which contains our list of pixels, and `labs`, which is our list of labels for each pixel---`0` for green background pixels and `1` for foreground pixels

In [12]:
# Build a list of pixels for both positive and negative examples.
alldat = np.concatenate((yesList,noList))

# labels
labs = np.concatenate((np.ones(len(yesList)), np.zeros(len(noList))))

Build a classifier to separate the background from the foreground.

We define a `loss` function.  The `loss` takes in our data `alldat`, our labels `labs`, and our current weights `w`.  Make adjustments to `w` based on this loss function.  To make a prediction, multiply `w` by `alldat`, and pass it through a sigmoid function to get our predictions `y`.  Then, compare our predictions `y` to their true labels `labs` using a squared loss, the sum of the squared difference between our true labels `labs`, and our predicted values `y`.

In [13]:
# Add an additional column to the data corresponding to
#  the offset parameter.
alldat = np.concatenate((alldat,np.ones((len(alldat),1))),1)

# Compute the loss of the rule specified by weights w with respect
#  to the data alldat labeled with labs
def loss(w, alldat, labs):
  # Compute a weighted sum for each instance
  h = np.matmul(alldat,w)
  # transform the sum using the sigmoid function
  y = 1/(1 + np.exp(-h))
  # take the difference between the labels and the output of the
  #  sigmoid, squared, then sum up over all instances to get the
  #  total loss.
  loss = np.sum((labs - y)**2)
  return(loss)

Train the model, by creating a `fit` function to fit the model to the data. Update the weights through gradient descent.

In [14]:
def fit(w,alldat,labs):
  # alpha represents how big of a step we’ll
  #  be taking in the direction of the derivative.
  #  It’s called the learning rate.
  alpha = 0.1

  # We'll stop searching when we're at a (near) local min
  done = False
  while not done:
    # Every 100 iterations or so, let’s
    #  take a peek at the weights, the learning
    #  rate, and the current loss
    if np.random.random() < 0.01: print(w, alpha, loss(w,alldat,labs))
    # The next few lines compute the gradient
    #  of the loss function. The details aren’t
    #  important right now.
    # delta_w is the change in the weights
    #  suggested by the gradient
    h = np.matmul(alldat,w)
    y = 1/(1 + np.exp(-h))
    delta_w = np.add.reduce(np.reshape((labs-y) * np.exp(-h)*y**2,(len(y),1)) * alldat)
    # if we take a step of size alpha and update
    #  the weights, we’ll get new weights neww.
    current_loss = loss(w,alldat,labs)
    alpha *= 2
    neww = w + alpha* delta_w
    while loss(neww,alldat,labs) >= current_loss and not done:
      alpha /= 2
      if alpha*max(abs(delta_w)) < 0.0001:
        done = True
        print(alpha,delta_w)
      else: neww = w + alpha* delta_w
    if not done: w = neww
  return(w)

w = np.random.random(4)
w = fit(w,alldat,labs)
print(w)

<ipython-input-13-43a48f8a219c>:11: RuntimeWarning: overflow encountered in exp
  y = 1/(1 + np.exp(-h))


[ 0.18334597 -0.1838335   0.83186319  0.51567514] 2.44140625e-05 48564.363224595945
[-0.87474947  0.07668323 -0.1783271   0.42942369] 3.0517578125e-06 507.77226527317634
[-0.87752763  0.08407091 -0.20889486  0.42331113] 6.103515625e-06 495.8377839872295
[-0.88328776  0.09717871 -0.24869324  0.41132478] 1.220703125e-05 486.10499937754594
[-0.88737659  0.10183956 -0.26892571  0.40304967] 6.103515625e-06 482.578372616331
[-0.89935188  0.11437686 -0.31021545  0.37933802] 1.220703125e-05 477.12836403526
[-0.90447007  0.1177554  -0.324012    0.36933359] 1.220703125e-05 475.620460637414
[-0.9053911   0.11859153 -0.32624807  0.3675404 ] 6.103515625e-06 475.35847223909764
[-0.91315335  0.12346176 -0.34389076  0.35247579] 1.220703125e-05 473.5573270338417
[-0.94480381  0.13852717 -0.39319062  0.29150701] 1.220703125e-05 468.589714072507
[-0.97247681  0.14588299 -0.41656275  0.237679  ] 2.44140625e-05 465.66868804329164
[-1.01240273  0.15022757 -0.43126166  0.15683332] 1.220703125e-05 462.0678474

Learned weights from four runs.

In [15]:
# [-1.63963568  0.11244618 -0.25229998 -1.83450268]
# [-1.43047609  0.11237203 -0.25146534 -1.86348854]
# [-0.90529042  0.11226296 -0.24705779 -2.05801113]
# [-1.13011592  0.11231782 -0.24929065 -1.95950121]